In [1]:
%load_ext autoreload
%autoreload 2

### Install SDG
```bash 
git clone https://github.com/Red-Hat-AI-Innovation-Team/sdg_hub.git
cd sdg_hub
pip install .[examples]
```
**⚠️ If you haven't already, run the document pre-processing notebook to create the seed data.**

In [2]:
from ai_tools.usecase.knowledge_tuning.knowledge_utils import create_knowledge_regular_ds, create_knowledge_pretraining_ds

/Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from pathlib import Path

WORKSPACE = Path.cwd().parent # Path to the workspace directory


OUTPUT_DIR= WORKSPACE / "output" / "step_02"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # Create the output directory if it doesn't exist


SEED_DATA_FILE = WORKSPACE / "output" / "step_01" / "final_seed_data.jsonl" # Path to the seed data file generated in step 1


if not SEED_DATA_FILE.exists():
    raise FileNotFoundError(f"\nNot a valid seed data ! {SEED_DATA_FILE}.\nPlease run step 1 to generate the seed data. \n(or) Provide the correct path to the seed data file.")

# CONFIGURE MODEL DETAILS HERE FOR THE FLOW
MODEL_NAME = "openai/llama-4-scout-17b-16e-w4a16"
API_KEY = ""   # Provide your API key here
ENDPOINT = "https://llama-4-scout-17b-16e-w4a16-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1"

In [4]:
# Third Party
from datasets import load_dataset

# First Party
from sdg_hub import Flow, FlowRegistry

In [5]:
# Required to run the flow with async mode
import nest_asyncio

nest_asyncio.apply()  

### Run SDG
- This will create knowledge flow from provided yaml file
- We will run this on small dataset for demo purposes
- For large scale generation, please use the python command provided in the next cell
- You can analyze the generated data to ensure the quality is similar to proivded QnA pairs

#### Discover the available generation flows

In [6]:
# Auto-discover all available flows (no setup needed!)
FlowRegistry.discover_flows()

# List available flows
flows = FlowRegistry.list_flows()
print(f"Available flows: {flows}")

# You can also search the flows by tag
qa_flows = FlowRegistry.search_flows(tag="question-generation")
print(f"QA flows: {qa_flows}")

[15:43:16] INFO     Discovered 5 flows                                                              ]8;id=475665;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/registry.py\registry.py]8;;\:]8;id=615449;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/registry.py#113\113]8;;\

[15:43:16] INFO     Discovered 5 flows                                                              ]8;id=4350;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/registry.py\registry.py]8;;\:]8;id=604594;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/registry.py#113\113]8;;\

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID               ┃ Name                  ┃ Author               ┃ Tags                  ┃ Description           ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ epic-jade-656    │ Extractive Summary    │ SDG Hub Contributors │ knowledge-tuning,     │ Generate extractive   │
│                  │ Knowledge Tuning      │                      │ document-internaliza… │ summary from the      │
│                  │ Dataset Generation    │                      │ question-generation,  │ input document. Each  │
│                  │ Flow                  │                      │ knowledge-extractive… │ document is first     │
│                  │                       │                      │ qa-pairs,             │ converted into list   │
│                  │                       │                      │ extractive-summaries  │ of knowledge segments │
│                  │                       │                      │                       │ for creating          │
│                  │                       │                      │                       │ extractive summary    │
│                  │                       │                      │                       │ and then annotated    │
│                  │                       │                      │                       │ with context,         │
│                  │                       │                      │                       │ relationship and      │
│                  │                       │                      │                       │ relevance. This is    │
│                  │                       │                      │                       │ then converted into   │
│                  │                       │                      │                       │ Question-Answer       │
│                  │                       │                      │                       │ pairs.                │
│ green-clay-812   │ Structured Text       │ SDG Hub Contributors │ text-analysis,        │ Multi-step pipeline   │
│                  │ Insights Extraction   │                      │ summarization, nlp,   │ for extracting        │
│                  │ Flow                  │                      │ structured-output,    │ structured insights   │
│                  │                       │                      │ insights,             │ from text including   │
│                  │                       │                      │ sentiment-analysis,   │ summary, keywords,    │
│                  │                       │                      │ entity-extraction,    │ entities, and         │
│                  │                       │                      │ keyword-extraction    │ sentiment analysis    │
│                  │                       │                      │                       │ combined into a JSON  │
│                  │                       │                      │                       │ output                │
│ heavy-heart-77   │ Key Facts Knowledge   │ SDG Hub Contributors │ knowledge-tuning,     │ Generating list of    │
│                  │ Tuning Dataset        │                      │ document-internaliza… │ atomic facts from a   │
│                  │ Generation Flow       │                      │ question-generation,  │ document and          │
│                  │                       │                      │ qa-pairs, key-facts   │ converting each       │
│                  │                       │                      │                       │ atomic fact into a QA │
│                  │                       │                      │                       │ pair. This flow will  │
│                  │                       │                      │                       │ generate 5 QA pairs   │
│                  │                       │            

Available flows: [{'id': 'green-clay-812', 'name': 'Structured Text Insights Extraction Flow'}, {'id': 'small-rock-799', 'name': 'Advanced Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}, {'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}, {'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}, {'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}]
QA flows: [{'id': 'small-rock-799', 'name': 'Advanced Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}, {'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}, {'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}, {'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}]


In [7]:
# We will use the "Advanced Document Grounded Question-Answer Generation Flow for Knowledge Tuning" flow.
# For loading the flow simply use the fullname to load it
flow_name = "Advanced Document Grounded Question-Answer Generation Flow for Knowledge Tuning"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)

[15:43:17] INFO     Loading flow from:                                                                  ]8;id=757786;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=968237;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#172\172]8;;\
                    /Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_            
                    Generation/.venv/lib/python3.12/site-packages/sdg_hub/flows/qa_generation/document_            
                    grounded_qa/multi_summary_qa/instructlab/flow.yaml                                             

[15:43:17] INFO     Loading flow from:                                                                  ]8;id=79309;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=319249;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#172\172]8;;\
                    /Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_            
                    Generation/.venv/lib/python3.12/site-packages/sdg_hub/flows/qa_generation/document_            
                    grounded_qa/multi_summary_qa/instructlab/flow.yaml                                             

#### Identify the recommended model and set the model config

In [8]:
flow.get_default_model()

'meta-llama/Llama-3.3-70B-Instruct'

In [9]:
flow.get_model_recommendations()

{'default': 'meta-llama/Llama-3.3-70B-Instruct',
 'compatible': ['microsoft/phi-4', 'mistralai/Mixtral-8x7B-Instruct-v0.1'],
 'experimental': []}

In [10]:
# You can dynamically change the model without having to change the flow yaml file.
# Configure the flow to use a vllm model hosted at localhost:8000/v1. 
flow.set_model_config(
    model=MODEL_NAME,
    api_base=ENDPOINT,
    api_key=API_KEY,
)

           INFO     Auto-detected 7 LLM blocks for configuration: ['eval_faithfulness',                 ]8;id=269390;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=157395;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#864\864]8;;\
                    'eval_relevancy', 'gen_atomic_facts', 'gen_detailed_summary',                                  
                    'gen_extractive_summary', 'knowledge_generation', 'verify_question']                           

           INFO     Auto-detected 7 LLM blocks for configuration: ['eval_faithfulness',                 ]8;id=211357;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=787201;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#864\864]8;;\
                    'eval_relevancy', 'gen_atomic_facts', 'gen_detailed_summary',                                  
                    'gen_extractive_summary', 'knowledge_generation', 'verify_question']                           

[15:43:17] INFO     Loaded LLM client for model 'openai/llama-4-scout-17b-16e-w4a16'           ]8;id=712133;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=976952;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

           INFO     Loaded LLM client for model 'openai/llama-4-scout-17b-16e-w4a16'           ]8;id=54011;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/client_manager.py\client_manager.py]8;;\:]8;id=392364;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/client_manager.py#61\61]8;;\

[15:43:17] INFO     Initialized LLMChatBlock 'gen_detailed_summary' with model                ]8;id=198543;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=790278;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'openai/llama-4-scout-17b-16e-w4a16'                                                           

...

           INFO     Initialized LLMChatBlock 'verify_question_llm_chat' with model            ]8;id=247645;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/llm_chat_block.py\llm_chat_block.py]8;;\:]8;id=24147;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/blocks/llm/llm_chat_block.py#265\265]8;;\
                    'openai/llama-4-scout-17b-16e-w4a16'                                                           

           INFO     Successfully configured 7 LLM blocks with: model:                                   ]8;id=637896;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=445232;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#903\903]8;;\
                    'openai/llama-4-scout-17b-16e-w4a16', api_base:                                                
                    'https://llama-4-scout-17b-16e-w4a16-maas-apicast-production.apps.prod.rhoai.rh-ais            
                    ervices-bu.com:443/v1', api_key: 1156ecc8851c8afdfb1011bf114d2ac5                              

           INFO     Successfully configured 7 LLM blocks with: model:                                   ]8;id=900785;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=949513;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#903\903]8;;\
                    'openai/llama-4-scout-17b-16e-w4a16', api_base:                                                
                    'https://llama-4-scout-17b-16e-w4a16-maas-apicast-production.apps.prod.rhoai.rh-ais            
                    ervices-bu.com:443/v1', api_key: 1156ecc8851c8afdfb1011bf114d2ac5                              

           INFO     Configured blocks: ['eval_faithfulness', 'eval_relevancy', 'gen_atomic_facts',      ]8;id=876704;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=999848;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#906\906]8;;\
                    'gen_detailed_summary', 'gen_extractive_summary', 'knowledge_generation',                      
                    'verify_question']                                                                             

           INFO     Configured blocks: ['eval_faithfulness', 'eval_relevancy', 'gen_atomic_facts',      ]8;id=330368;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=922198;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#906\906]8;;\
                    'gen_detailed_summary', 'gen_extractive_summary', 'knowledge_generation',                      
                    'verify_question']                                                                             

In [11]:
# Load the seed data
number_of_samples = 2

ds = load_dataset('json', data_files=f"{SEED_DATA_FILE}", split='train')
ds = ds.shuffle(seed=42).select(range(number_of_samples))

In [12]:
# Generate data
generated_data = flow.generate(ds)

           INFO     Starting flow 'Advanced Document Grounded Question-Answer Generation Flow for       ]8;id=525683;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=297260;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#515\515]8;;\
                    Knowledge Tuning' v1.0.0 with 2 samples across 18 blocks                                       

           INFO     Starting flow 'Advanced Document Grounded Question-Answer Generation Flow for       ]8;id=341260;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=538418;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#515\515]8;;\
                    Knowledge Tuning' v1.0.0 with 2 samples across 18 blocks                                       

           INFO     Executing block 1/18: duplicate_document_col (DuplicateColumnsBlock)                ]8;id=315006;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=149927;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#670\670]8;;\

           INFO     Executing block 1/18: duplicate_document_col (DuplicateColumnsBlock)                ]8;id=746620;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=110332;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#670\670]8;;\

╭──────────────────────────────────────────── duplicate_document_col ─────────────────────────────────────────────╮
│ 📊 Processing Input Data                                                                                        │
│ Block Type: DuplicateColumnsBlock                                                                               │
│ Input Rows: 2                                                                                                   │
│ Input Columns: 11                                                                                               │
│ Column Names: document, document_outline, document_title, domain, icl_document, icl_query_1, icl_response_1,    │
│ icl_query_2, icl_response_2, icl_query_3, icl_response_3                                                        │
│ Expected Output Columns: base_document                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

[15:44:33] INFO     Block 'verify_question' completed successfully: 47 samples, 35 columns              ]8;id=23793;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=775700;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#732\732]8;;\

           INFO     Block 'verify_question' completed successfully: 47 samples, 35 columns              ]8;id=832290;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=705004;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#732\732]8;;\

╭────────── Advanced Document Grounded Question-Answer Generation Flow for Knowledge Tuning - Complete ───────────╮
│                                        Flow Execution Summary                                                   │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓           │
│ ┃ Block Name           ┃ Type            ┃   Duration ┃     Rows     ┃     Columns     ┃   Status   ┃           │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩           │
│ │ duplicate_document_… │ DuplicateColum… │      0.01s │    2 → 2     │       +1        │     ✓      │           │
│ │ detailed_summary_pr… │ PromptBuilderB… │      0.01s │    2 → 2     │       +1        │     ✓      │           │
│ │ gen_detailed_summary │ LLMChatBlock    │      5.21s │    2 → 2     │       +1        │     ✓      │           │
│ │ parse_detailed_summ… │ TextParserBlock │      0.02s │    2 → 2     │       +1        │     ✓      │           │
│ │ atomic_facts_prompt  │ PromptBuilderB… │      0.02s │    2 → 2     │       +1        │     ✓      │           │
│ │ gen_atomic_facts     │ LLMChatBlock    │      4.58s │    2 → 2     │       +1        │     ✓      │           │
│ │ parse_atomic_facts   │ TextParserBlock │      0.02s │    2 → 2     │       +1        │     ✓      │           │
│ │ extractive_summary_… │ PromptBuilderB… │      0.02s │    2 → 2     │       +1        │     ✓      │           │
│ │ gen_extractive_summ… │ LLMChatBlock    │      3.69s │    2 → 2     │       +1        │     ✓      │           │
│ │ parse_extractive_su… │ TextParserBlock │      0.03s │    2 → 2     │       +1        │     ✓      │           │
│ │ melt_summary_columns │ MeltColumnsBlo… │      0.04s │    2 → 8     │      +2/-4      │     ✓      │           │
│ │ rename_to_document_… │ RenameColumnsB… │      0.00s │    8 → 8     │      +1/-1      │     ✓      │           │
│ │ knowledge_generatio… │ PromptBuilderB… │      0.02s │    8 → 8     │       +1        │     ✓      │           │
│ │ knowledge_generation │ LLMChatBlock    │     19.09s │    8 → 8     │       +1        │     ✓      │           │
│ │ parse_knowledge_gen… │ TextParserBlock │      0.03s │    8 → 72    │       +2        │     ✓      │           │
│ │ eval_faithfulness    │ EvaluateFaithf… │     12.72s │   72 → 71    │       +4        │     ✓      │           │
│ │ eval_relevancy       │ EvaluateReleva… │     12.68s │   71 → 62    │       +4        │     ✓      │           │
│ │ verify_question      │ VerifyQuestion… │     17.47s │   62 → 47    │       +4        │     ✓      │           │
│ ├──────────────────────┼─────────────────┼────────────┼──────────────┼─────────────────┼────────────┤           │
│ │ TOTAL                │ 18 blocks       │     75.66s │   47 final   │    35 final     │   18/18    │           │
│ └──────────────────────┴─────────────────┴────────────┴──────────────┴─────────────────┴────────────┘           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

           INFO     Flow 'Advanced Document Grounded Question-Answer Generation Flow for Knowledge      ]8;id=664453;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=240196;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#620\620]8;;\
                    Tuning' completed successfully: 47 final samples, 35 final columns                             

           INFO     Flow 'Advanced Document Grounded Question-Answer Generation Flow for Knowledge      ]8;id=323034;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=227820;file:///Users/scharan/projects/red-hat-ai-examples/examples/knowledge-tuning/02_Knowledge_Generation/.venv/lib/python3.12/site-packages/sdg_hub/core/flow/base.py#620\620]8;;\
                    Tuning' completed successfully: 47 final samples, 35 final columns                             

### Converting the generated data into training format

In [13]:
from ai_tools.usecase.knowledge_tuning.knowledge_utils import create_knowledge_regular_ds, create_knowledge_pretraining_ds

from datasets import concatenate_datasets



# Create Pretraining Knowledge Dataset (Also known as Phase 0.7/Phase 7)
instructlab_phase_1_ds = create_knowledge_pretraining_ds(generated_data)
instructlab_phase_1_ds.to_json(f'{OUTPUT_DIR}/instructlab_phase_1_ds.jsonl', orient='records', lines=True)

# Create Regular Knowledge Dataset (Also known as Phase 1.0/Phase 10)
instructlab_phase_2_ds = create_knowledge_regular_ds(generated_data)

# Mix the pre-computed skills with the regular knowledge dataset. If more than one dataset were generated simply add those in this concatenation stage.
# If you have any generated instruction data, that can be also mixed in this stage. If you only have generated skills phase 07 generation and training can be skipped.
instructlab_phase_2_ds.to_json(f'{OUTPUT_DIR}/instructlab_phase_2_ds.jsonl', orient='records', lines=True)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 376.47ba/s]


312751

In [ ]:
# If you have any other instruction tuning datasets you can mix with phase 2 dataset.
instruction_tuning_dataset_path = "<Your instruction tuning dataset path>"
instruction_tuning_dataset = load_dataset('json', data_files=instruction_tuning_dataset_path, split='train')
instructlab_phase_2_ds = concatenate_datasets([instructlab_phase_2_ds, instruction_tuning_dataset])
instructlab_phase_2_ds.to_json(f'{OUTPUT_DIR}/instructlab_phase_2_ds.jsonl', orient='records', lines=True)